In [1]:
import pandas as pd

In [3]:
# read the csv and put the data into a df
df = pd.read_csv("nba_games.csv", index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: 'nba_games.csv'

In [4]:
# sort the values by date (earliest games to latest games)
df = df.sort_values("date")
# index is now out of order so we have to reset the index, drop=True means it will drop the previous index instead of making it a column
df = df.reset_index(drop=True)
# delete these columns with the keyword "del" bc they are duplicates of other columns
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

NameError: name 'df' is not defined

In [5]:
# create a new column for each game named target which represents whether the team won the next game or not
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team
# group each game by team, so all wizards games are together
df = df.groupby("team", group_keys=False).apply(add_target)

NameError: name 'df' is not defined

In [6]:
# this command is for the target column and specifies if the value is null, then to assign it the value 2
df["target"][pd.isnull(df["target"])] = 2

NameError: name 'df' is not defined

In [7]:
# converts all the bool values that represent whether the next game is a win or loss into integers.
# the errors=ignore part is for the command we did before, since the null values are already ints (2), we will ignore the error for converting those
df["target"] = df["target"].astype(int, errors="ignore")

NameError: name 'df' is not defined

In [8]:
# pulls all the values that are null
nulls = pd.isnull(df)

NameError: name 'df' is not defined

In [9]:
# sums all the columns that are null
nulls = nulls.sum()

NameError: name 'nulls' is not defined

In [10]:
# assigns all the columns that are null to null
nulls = nulls[nulls > 0]

NameError: name 'nulls' is not defined

In [19]:
# if the columnns are in null, then do not include them in valid columns (~ is the negation sign for pandas)
valid_columns = df.columns[~df.columns.isin(nulls.index)]

NameError: name 'df' is not defined

In [20]:
# copies the df with the valid columns into df (copy is used so an error doesn't show up with pandas)
df = df[valid_columns].copy()

NameError: name 'df' is not defined

In [18]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
# ridge regression to do classification to see whether the team would win or not
from sklearn.linear_model import RidgeClassifier


rr = RidgeClassifier(alpha=1)

split = TimeSeriesSplit(n_splits=3)

# only selecting 30 features(columns/stats) to train the model, 
# direction forward means since its sequential, the model will pick the features that improves accuracy the most, until we get to 30 features selected
sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction="forward", cv=split)

In [17]:
# before running the feature selection, we need to scale the columns
# meta data columns that we don't need to scale
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]

In [21]:
# the selected columns are the ones not in removed columns
selected_columns = df.columns[~df.columns.isin(removed_columns)]

NameError: name 'df' is not defined

In [22]:
from sklearn.preprocessing import MinMaxScaler

# scales all the data in between 0 and 1 so ridgeregression model can work better
scaler = MinMaxScaler()
df[selected_columns] = scalar.fit_transform(df[selected_columns])

NameError: name 'scalar' is not defined

In [24]:
# this picks the 30 best features for picking this target
sfs.fit(df[selected_columns], df["target"])

NameError: name 'df' is not defined

In [25]:
predictors = list(selected_columns[sfs.get_support()])

NameError: name 'selected_columns' is not defined

In [26]:
# we have to use previous season data to predict current season data
# start=2 means that we need at least 2 seasons worth of data to predict the next seasons data
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []
    seasons = sorted(data["season"].unique())

    ''' 
    since we're starting from 2016, the first iteration of the loop would be 2018.
    the training data would be 2016, and 2017 since we wanted to use at least 2 years 
    of data to make predictions
    '''
    for i in range(start, len(seasons), step):
        season = seasons[i]

        # all the data that comes before the current season
        train = data[data["season"] < season]
        # current season
        test = data[data["season"] == season]

        # put the training data into the model and train it to predict the target value
        model.fit(train[predictors], train["target"])
        # model predicts the target value on the test data. 
        # model.predict returns a numpy array which we don't want
        preds = model.predict(test[predictors])
        # convert numpy array to pd.series
        preds = pd.Series(preds, index=test.index)

        # combine the two dataframes together 
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]

        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [28]:
predictions = backtest(df, rr, predictors)

NameError: name 'df' is not defined

In [29]:
from sklearn.metrics import accuracy_score

predictions = predictions[predictions["actual"] != 2]
accuracy_score(predictions["actual"], predictions["prediction"])

NameError: name 'predictions' is not defined